In [9]:
import tensorflow as tf
assert tf.test.is_gpu_available()
assert tf.test.is_built_with_cuda()
tf.test.gpu_device_name()

'/device:GPU:0'

In [37]:
import csv

def extract(json, out_dict):
    for item in json['data']:
        out_dict['title'].append(item['title'])
        out_dict['score'].append(item['score'])
        out_dict['id'].append(item['id'])
        out_dict['comments'].append(item['num_comments'])
        out_dict['created'].append(item['created_utc'])
        try:
            out_dict['body'].append(item['selftext'])
        except KeyError:
            out_dict['body'].append('')
        try:
            out_dict['subreddit_subscribers'].append(item['subreddit_subscribers'])
        except KeyError:
            out_dict['subreddit_subscribers'].append('')

def scrape(out_dict, end, iterations, interval, interval_divisor):
    error_log = []
    for i in range(iterations):
        if not i%1000: print("Iteration: ",i)
        begin = end - interval

        url = 'https://api.pushshift.io/reddit/search/submission/?q=cryptocurrency&size=500'
        url += '&before={}'.format(int(end.timestamp()))
        url += '&after={}'.format(int(begin.timestamp()))

        try:
            json = requests.get(url).json()
            
        except ValueError:
            try:
                json = requests.get(url).json()
                
            except ValueError:
                error_log.append(url)
                

        if len(json['data']) == 500:
            print(end)
            scrape(out_dict, end, interval_divisor, (interval/interval_divisor), interval_divisor)

        else:
#             for item in json['data']:
#                 writer.writerow(list(item.values()))
          extract(json, out_dict)

        end = begin
        
    return error_log

In [22]:
import datetime as dt
import requests
import pandas as pd

data_dict = {"title":[],
                "score":[],
                "id":[],
                "comments": [],
                "created": [],
                "body":[],
                "subreddit_subscribers":[]}

end = dt.datetime.today()

interval = dt.timedelta(hours=6)

interval_divisor = 6

iterations = 9120

# with open('one_json.csv', 'w') as file:
#     writer = csv.writer(file)
error_log = scrape(data_dict, end, iterations, interval, interval_divisor)

data = pd.DataFrame(data_dict)

data.to_csv('reddit.csv', index=False)

Iteration:  0
2019-06-18 19:47:55.202275
Iteration:  0
2019-06-18 13:47:55.202275
Iteration:  0
Iteration:  1000
2018-09-25 19:47:55.202275
Iteration:  0
2018-07-31 13:47:55.202275
Iteration:  0
2018-07-31 07:47:55.202275
Iteration:  0
2018-07-31 01:47:55.202275
Iteration:  0
2018-07-30 19:47:55.202275
Iteration:  0
2018-07-30 13:47:55.202275
Iteration:  0
2018-07-29 01:47:55.202275
Iteration:  0
2018-07-28 19:47:55.202275
Iteration:  0
2018-07-28 13:47:55.202275
Iteration:  0
2018-07-27 19:47:55.202275
Iteration:  0
Iteration:  2000
2018-01-31 13:47:55.202275
Iteration:  0
2018-01-17 19:47:55.202275
Iteration:  0
2018-01-17 01:47:55.202275
Iteration:  0
2018-01-16 19:47:55.202275
Iteration:  0
2018-01-12 19:47:55.202275
Iteration:  0
2018-01-08 19:47:55.202275
Iteration:  0
Iteration:  3000
Iteration:  4000
Iteration:  5000
Iteration:  6000
Iteration:  7000
Iteration:  8000
Iteration:  9000


ValueError: arrays must all be same length

In [26]:
for item in data_dict:
    print(item, len(data_dict[item]))

comments 577158
subreddit 756117
subreddit_subscribers 577158
score 577158
created 577158
id 577158
title 577158
body 577158


In [31]:
new_dict = {"title":[],
                "id":[],
                "comments": [],
                "created": [],
                "body":[],
                "score":[],
                "subreddit_subscribers":[]}

for item in data_dict:
    if item != "subreddit":
        new_dict[item] = data_dict[item]

In [32]:
data = pd.DataFrame(new_dict)

data.to_csv('reddit.csv', index=False)

In [39]:
d = {"title":[],
        "id":[],
        "comments": [],
        "created": [],
        "body":[],
        "score":[],
        "subreddit_subscribers":[]}
for item in error_log:
    json = requests.get(item).json()
    extract(json, d)

In [42]:
with open('reddit.csv', 'a') as file:
    writer = csv.writer(file)
    for i in range(len(d['comments'])):
        line = []
        for item in d:
            line.append(d[item][i])
        writer.writerow(line)